In [ ]:
import pandas as pd
import datetime as dt

# Load JSON data
df = pd.read_json("C:/Users/Dileep Sathya/Desktop/screener/artifacts/hist_data.json")

# Ensure 'date' is datetime
df['date'] = pd.to_datetime(df['date'])

# Add year column
df['year'] = df['date'].dt.year

# Create previous day features
df['prev_day_open'] = df.groupby('symbol')['open'].shift(1)
df['prev_day_high'] = df.groupby('symbol')['high'].shift(1)
df['prev_day_low'] = df.groupby('symbol')['low'].shift(1)
df['prev_day_close'] = df.groupby('symbol')['close'].shift(1)

df = df.dropna()

# Calculate previous day features
df['prev_day_range'] = abs((df['prev_day_high'] - df['prev_day_low']) / df['prev_day_low']) * 100
df['prev_day_body_range'] = abs((df['prev_day_close'] - df['prev_day_open']) / df['prev_day_open']) * 100

df = df.dropna()

# Filter for specific symbol



# Apply eligibility condition
df['eligibility'] = (df['prev_day_range'] < 1.75) & (df['prev_day_body_range'] < 0.5)

# Current day ranges
df['range'] = abs((df['high'] - df['low']) / df['low']) * 100
df['body_range'] = abs((df['close'] - df['open']) / df['open']) * 100

# Drop unused columns
df = df.drop(columns=[
    'open', 'high', 'low', 'close', 'volume', 'year',
    'prev_day_high', 'prev_day_low', 'prev_day_close', 'prev_day_open'
])

# Filter for past 366 days only
today = pd.to_datetime("2025-06-16")
start_date = today - pd.Timedelta(days=183)
df = df[(df['date'] >= start_date) & (df['date'] <= today)]
df = df.sort_values('date')

# Final filtered DataFrame based on eligibility
fil_df = df[df['eligibility'] == True]


In [2]:
fil_df

,date,symbol,prev_day_range,prev_day_body_range,eligibility,range,body_range


In [13]:
df['prev_day_body_range'].describe()

count    123.000000
mean       1.393091
std        1.099625
min        0.019011
25%        0.671495
50%        1.121272
75%        1.869067
max        6.048387
Name: prev_day_body_range, dtype: float64

In [3]:
df['prev_day_range'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: prev_day_range, dtype: float64

In [7]:
df = pd.read_json("C:/Users/Dileep Sathya/Desktop/screener/artifacts/hist_data.json")

# Ensure 'date' is datetime
df['date'] = pd.to_datetime(df['date'])
df['range'] = abs((df['high'] - df['low']) / df['low']) * 100
df['body_range'] = abs((df['close'] - df['open']) / df['open']) * 100
df['eligibility'] = (df['range'] < 1.75) & (df['body_range'] < 0.5)

df[df['eligibility'] == True]

,date,open,high,low,close,volume,symbol,range,body_range,eligibility
5,2025-06-16,287.95,290.5,286.30,288.65,11067299,POWERGRID,1.466993,0.243098,True
18,2025-06-16,792.50,795.4,786.15,792.50,8085001,SBIN,1.176620,0.000000,True
